In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import * 
from pyspark.sql.functions import regexp_replace,split,substring

In [2]:
spark = SparkSession.builder \
.master("yarn") \
.appName("DataframeWriter") \
.enableHiveSupport() \
.getOrCreate()

2023-02-23 17:40:36,519 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-02-23 17:40:44,894 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


# Read Data

In [4]:
df = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/dirty_store_transactions.csv")

In [5]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import  *

In [6]:
programmatical_schema = StructType([
        StructField("STORE_ID",StringType(),True),
        StructField("STORE_LOCATION",StringType(),True),
        StructField("PRODUCT_CATEGORY",StringType(),True),
        StructField("PRODUCT_ID",StringType(),True),
        StructField("MRP",StringType(),True),
        StructField("CP",StringType(),True),
        StructField("DISCOUNT",StringType(),True),
        StructField("SP",StringType(),True),
        StructField("Date",DateType(),True)
    ])

In [7]:
df2 = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.schema(programmatical_schema) \
.load("file:///home/train/datasets/dirty_store_transactions.csv")

In [8]:
symbol_pattern = r"[^\w\s]"

In [9]:
df3 = df2.withColumn("STORE_LOCATION", regexp_replace("STORE_LOCATION", symbol_pattern, ""))

In [10]:
df4 = df3.withColumn("PRODUCT_ID", regexp_replace("PRODUCT_ID", symbol_pattern, ""))


In [11]:
df4.show(n=15, truncate=False)


+--------+--------------+----------------+----------+---+------+--------+------+----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|CP    |DISCOUNT|SP    |Date      |
+--------+--------------+----------------+----------+---+------+--------+------+----------+
|YR7220  |New York      |Electronics     |12254943  |$31|$20.77|$1.86   |$29.14|2019-11-26|
|YR7220  |New York      |Furniture       |72619323C |$15|$9.75 |$1.5    |$13.5 |2019-11-26|
|YR7220  |New York      |Electronics     |34161682B |$88|$62.48|$4.4    |$83.6 |2019-11-26|
|YR7220  |New York      |Kitchen         |79411621  |$91|$58.24|$3.64   |$87.36|2019-11-26|
|YR7220  |New York      |Fashion         |39520263T |$85|$51   |$2.55   |$82.45|2019-11-26|
|YR7220  |New York      |Kitchen         |93809204  |$37|$24.05|$0.74   |$36.26|2019-11-26|
|YR7220  |New York      |Cosmetics       |86610412D |$80|$48.8 |$6.4    |$73.6 |2019-11-26|
|YR7220  |New York      |Kitchen         |52503356  |$71|$42.6 |$5.68   |$65.32|

In [12]:
df4.limit(5).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,New York,Furniture,72619323C,$15,$9.75,$1.5,$13.5,2019-11-26
2,YR7220,New York,Electronics,34161682B,$88,$62.48,$4.4,$83.6,2019-11-26
3,YR7220,New York,Kitchen,79411621,$91,$58.24,$3.64,$87.36,2019-11-26
4,YR7220,New York,Fashion,39520263T,$85,$51,$2.55,$82.45,2019-11-26


In [13]:
from pyspark.sql.functions import split, col


df4 = df4.withColumn('Money Unity', substring('CP', 1, 1))

In [14]:

df4 = df4.withColumn("CP", split(col("CP"), "\$").getItem(1))
df4 = df4.withColumn("DISCOUNT", split(col("DISCOUNT"), "\$").getItem(1))
df4 = df4.withColumn("SP", split(col("SP"), "\$").getItem(1))
df4 = df4.withColumn("MRP", split(col("MRP"), "\$").getItem(1))

In [15]:
df4.show()

+--------+--------------+----------------+----------+---+-----+--------+-----+----------+-----------+
|STORE_ID|STORE_LOCATION|PRODUCT_CATEGORY|PRODUCT_ID|MRP|   CP|DISCOUNT|   SP|      Date|Money Unity|
+--------+--------------+----------------+----------+---+-----+--------+-----+----------+-----------+
|  YR7220|      New York|     Electronics|  12254943| 31|20.77|    1.86|29.14|2019-11-26|          $|
|  YR7220|      New York|       Furniture| 72619323C| 15| 9.75|     1.5| 13.5|2019-11-26|          $|
|  YR7220|     New York |     Electronics| 34161682B| 88|62.48|     4.4| 83.6|2019-11-26|          $|
|  YR7220|      New York|         Kitchen|  79411621| 91|58.24|    3.64|87.36|2019-11-26|          $|
|  YR7220|      New York|         Fashion| 39520263T| 85|   51|    2.55|82.45|2019-11-26|          $|
|  YR7220|      New York|         Kitchen|  93809204| 37|24.05|    0.74|36.26|2019-11-26|          $|
|  YR7220|      New York|       Cosmetics| 86610412D| 80| 48.8|     6.4| 73.6|2019

In [16]:
spark.sql("show databases").show()

2023-02-23 17:24:48,605 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2023-02-23 17:24:48,606 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist


+---------+
|namespace|
+---------+
|bookstore|
|  company|
|  default|
|    test1|
+---------+



In [ ]:
spark.sql("create database if not exists test1;")


In [18]:
df4.write \
.format("orc") \
.mode("overwrite") \
.saveAsTable("test1.clean_transactions")

2023-02-23 17:28:11,147 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
2023-02-23 17:28:11,273 WARN conf.HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
2023-02-23 17:28:11,273 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2023-02-23 17:28:11,273 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [19]:
df_from_orc = spark.read.orc("test1.clean_transactions")

In [20]:
df_from_orc.limit(2).toPandas()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date,Money Unity
0,YR7220,New York,Electronics,12254943,31,20.77,1.86,29.14,2019-11-26,$
1,YR7220,New York,Furniture,72619323C,15,9.75,1.5,13.5,2019-11-26,$
